In [1]:
import pandas as pd

In [2]:
agg_df = pd.read_csv('BRFSS_agg_selected.csv')
agg_df.head()

/var/folders/9j/2d4wrxnn5qnbxbd13xlv8tp80000gn/T/ipykernel_37495/585659718.py:1: DtypeWarning: Columns (13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_df = pd.read_csv('BRFSS_agg_selected.csv')


,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
0,2012,GU,Guam,Demographics,Age,What is your age?,55-64 years,Overall,Overall,296,...,NaN,BRFSS,CLASS05,Topic02,66,BO1,CAT1,AGE,RESP014,"(13.444304, 144.793731)"
1,2012,GU,Guam,Demographics,Age,What is your age?,35-44 years,Female,Gender,272,...,NaN,BRFSS,CLASS05,Topic02,66,SEX2,CAT2,AGE,RESP012,"(13.444304, 144.793731)"
2,2013,GU,Guam,Demographics,Marital Status,What is your marital status?,Partnered,H.S. or G.E.D.,Education Attained,48,...,NaN,BRFSS,CLASS05,Topic38,66,EDUCA2,CAT5,MARITAL,RESP038,"(13.444304, 144.793731)"
3,2013,PR,Puerto Rico,Demographics,Age,What is your age?,25-34 years,Some post-H.S.,Education Attained,183,...,NaN,BRFSS,CLASS05,Topic02,72,EDUCA3,CAT5,AGE,RESP011,"(18.220833, -66.590149)"
4,2011,GU,Guam,Demographics,Age,What is your age?,45-54 years,Hispanic,Race/Ethnicity,29,...,NaN,BRFSS,CLASS05,Topic02,66,RACE3,CAT4,AGE,RESP013,"(13.444304, 144.793731)"


In [3]:
def brfss_extract(question_id, breakout):
    question_df = agg_df[agg_df['QuestionID'] == question_id]
    breakout_df = question_df[question_df['Break_Out_Category'] == breakout]

    if breakout == 'Overall':
        kept_cols = ['Year', 'Locationabbr', 'Question', 'Response', 'Data_value', 'QuestionID', 'ResponseID']
        sorted_df = breakout_df[kept_cols].sort_values(by=['Year', 'Locationabbr', 'ResponseID'])
    else:
        kept_cols = ['Year', 'Locationabbr', 'Question', 'Response', 'Data_value', 'Break_Out', 'QuestionID', 'ResponseID']
        sorted_df = breakout_df[kept_cols].sort_values(by=['Year', 'Locationabbr', 'ResponseID', 'Break_Out'])

    output_df = sorted_df.reset_index().drop(columns=['index'])
    
    return output_df

In [13]:
ecig_2016_2017 = brfss_extract('_ECIGSTS', 'Overall')
ecig_2016_2017.head()

,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID
0,2016,AK,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
1,2016,AK,E-cigarette use status,Current E-cigarette user - some days,2.8,_ECIGSTS,RESP216
2,2016,AK,E-cigarette use status,Former E-cigarette user,20.6,_ECIGSTS,RESP217
3,2016,AK,E-cigarette use status,Never used E-cigarettes,75.3,_ECIGSTS,RESP218
4,2016,AL,E-cigarette use status,Current E-cigarette user - every day,1.6,_ECIGSTS,RESP215


In [14]:
#need to swap RESP217 and RESP218 for all states except PR, HI, UT, OR, IL, CO, AK, NC

ecig_2021 = brfss_extract('ECIGNOW1', 'Overall')

# States to exclude
exclude_states = ['PR', 'HI', 'UT', 'OR', 'IL', 'CO', 'AK', 'NC']

# Filter out the rows for the excluded states
filtered_df = ecig_2021[~ecig_2021['Locationabbr'].isin(exclude_states)]

# Get the indices of RESP217 and RESP218
resp217_indices = filtered_df[filtered_df['ResponseID'] == 'RESP217'].index
resp218_indices = filtered_df[filtered_df['ResponseID'] == 'RESP218'].index

# Swap the Data_value for RESP217 and RESP218
filtered_df.loc[resp217_indices, 'Data_value'], filtered_df.loc[resp218_indices, 'Data_value'] = filtered_df.loc[resp218_indices, 'Data_value'].values, filtered_df.loc[resp217_indices, 'Data_value'].values

# Update the original dataframe
ecig_2021.update(filtered_df)

ecig_2021


,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID
0,2021,AK,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,3.0,ECIGNOW1,RESP215
1,2021,AK,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - some days,3.2,ECIGNOW1,RESP216
2,2021,AK,Do you now use e-cigarettes or other electroni...,Former E-cigarette user,24.4,ECIGNOW1,RESP217
3,2021,AK,Do you now use e-cigarettes or other electroni...,Never used E-cigarettes,69.4,ECIGNOW1,RESP218
4,2021,AL,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,4.4,ECIGNOW1,RESP215
...,...,...,...,...,...,...,...
215,2021,WV,Do you now use e-cigarettes or other electroni...,Never used E-cigarettes,87.7,ECIGNOW1,RESP218
216,2021,WY,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,4.1,ECIGNOW1,RESP215
217,2021,WY,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - some days,3.4,ECIGNOW1,RESP216
218,2021,WY,Do you now use e-cigarettes or other electroni...,Former E-cigarette user,9.7,ECIGNOW1,RESP217


In [15]:
ecig_2022 = brfss_extract('ECIGNOW2', 'Overall')
ecig_2022.head()

,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID
0,2022,AK,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,3.4,ECIGNOW2,RESP215
1,2022,AK,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - some days,3.4,ECIGNOW2,RESP216
2,2022,AK,Do you now use e-cigarettes or other electroni...,Former E-cigarette user,24.7,ECIGNOW2,RESP217
3,2022,AK,Do you now use e-cigarettes or other electroni...,Never used E-cigarettes,68.5,ECIGNOW2,RESP218
4,2022,AL,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,4.7,ECIGNOW2,RESP215


In [16]:
#Combining all years
ecig_df = pd.concat([ecig_2016_2017, ecig_2021, ecig_2022], ignore_index=True)
ecig_df

,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID
0,2016,AK,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
1,2016,AK,E-cigarette use status,Current E-cigarette user - some days,2.8,_ECIGSTS,RESP216
2,2016,AK,E-cigarette use status,Former E-cigarette user,20.6,_ECIGSTS,RESP217
3,2016,AK,E-cigarette use status,Never used E-cigarettes,75.3,_ECIGSTS,RESP218
4,2016,AL,E-cigarette use status,Current E-cigarette user - every day,1.6,_ECIGSTS,RESP215
...,...,...,...,...,...,...,...
883,2022,WV,Do you now use e-cigarettes or other electroni...,Never used E-cigarettes,72.4,ECIGNOW2,RESP218
884,2022,WY,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,4.0,ECIGNOW2,RESP215
885,2022,WY,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - some days,4.3,ECIGNOW2,RESP216
886,2022,WY,Do you now use e-cigarettes or other electroni...,Former E-cigarette user,22.4,ECIGNOW2,RESP217


In [17]:
#Null Check
nan_rows = ecig_df[ecig_df['Data_value'].isnull()]
nan_rows

,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID
0,2016,AK,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
28,2016,DC,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
160,2016,PR,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
200,2016,VI,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
201,2016,VI,E-cigarette use status,Current E-cigarette user - some days,NaN,_ECIGSTS,RESP216
384,2017,PR,E-cigarette use status,Current E-cigarette user - every day,NaN,_ECIGSTS,RESP215
640,2021,VI,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,NaN,ECIGNOW1,RESP215
641,2021,VI,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - some days,NaN,ECIGNOW1,RESP216
692,2022,DC,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,NaN,ECIGNOW2,RESP215
864,2022,VI,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,NaN,ECIGNOW2,RESP215


In [18]:
#Removing VI, because too many missing years
ecig_df = ecig_df[ecig_df['Locationabbr'] != 'VI']

# Function to fill missing values
def fill_missing_values(group):
    total = group['Data_value'].sum()
    missing = group['Data_value'].isna()
    if missing.sum() == 1:
        group.loc[missing, 'Data_value'] = 100 - total
    return group

# Apply the function to each group
ecig_clean_df = ecig_df.groupby(['Year', 'Locationabbr']).apply(fill_missing_values).reset_index(drop=True)
ecig_clean_df


/var/folders/9j/2d4wrxnn5qnbxbd13xlv8tp80000gn/T/ipykernel_37495/3207802330.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ecig_clean_df = ecig_df.groupby(['Year', 'Locationabbr']).apply(fill_missing_values).reset_index(drop=True)


,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID
0,2016,AK,E-cigarette use status,Current E-cigarette user - every day,1.3,_ECIGSTS,RESP215
1,2016,AK,E-cigarette use status,Current E-cigarette user - some days,2.8,_ECIGSTS,RESP216
2,2016,AK,E-cigarette use status,Former E-cigarette user,20.6,_ECIGSTS,RESP217
3,2016,AK,E-cigarette use status,Never used E-cigarettes,75.3,_ECIGSTS,RESP218
4,2016,AL,E-cigarette use status,Current E-cigarette user - every day,1.6,_ECIGSTS,RESP215
...,...,...,...,...,...,...,...
871,2022,WV,Do you now use e-cigarettes or other electroni...,Never used E-cigarettes,72.4,ECIGNOW2,RESP218
872,2022,WY,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - every day,4.0,ECIGNOW2,RESP215
873,2022,WY,Do you now use e-cigarettes or other electroni...,Current E-cigarette user - some days,4.3,ECIGNOW2,RESP216
874,2022,WY,Do you now use e-cigarettes or other electroni...,Former E-cigarette user,22.4,ECIGNOW2,RESP217


In [20]:
#Null Check
nan_rows = ecig_clean_df[ecig_clean_df['Data_value'].isnull()]
nan_rows

,Year,Locationabbr,Question,Response,Data_value,QuestionID,ResponseID


In [21]:
# Separate the data into smokes and doesn't smoke
smokes = ecig_clean_df[ecig_clean_df['ResponseID'].isin(['RESP215', 'RESP216'])] \
    .groupby(['Year', 'Locationabbr'])['Data_value'] \
    .sum() \
    .reset_index() \
    .rename(columns={'Data_value': 'smoke_pct'})

doesnt_smoke = ecig_clean_df[ecig_clean_df['ResponseID'].isin(['RESP217', 'RESP218'])] \
    .groupby(['Year', 'Locationabbr'])['Data_value'] \
    .sum() \
    .reset_index() \
    .rename(columns={'Data_value': 'dont_smoke_pct'})

# Merge the two data frames together
ecig_by_state_2016_2017_2021_2022 = pd.merge(smokes, doesnt_smoke, on=['Year', 'Locationabbr'], how='left')
ecig_by_state_2016_2017_2021_2022 = ecig_by_state_2016_2017_2021_2022.rename(columns={'Locationabbr': 'State'})
ecig_by_state_2016_2017_2021_2022.head()


,Year,State,smoke_pct,dont_smoke_pct
0,2016,AK,4.1,95.9
1,2016,AL,5.2,94.9
2,2016,AR,5.8,94.2
3,2016,AZ,5.2,94.8
4,2016,CA,3.2,96.7


In [22]:
#Rounding the values to one decimal
ecig_by_state_2016_2017_2021_2022['smoke_pct'] = ecig_by_state_2016_2017_2021_2022['smoke_pct'].round(1)
ecig_by_state_2016_2017_2021_2022['dont_smoke_pct'] = ecig_by_state_2016_2017_2021_2022['dont_smoke_pct'].round(1)

In [23]:
csv_file_path = 'ecig_by_state_2016_2017_2021_2022.csv'
ecig_by_state_2016_2017_2021_2022.to_csv(csv_file_path, index=False)